# 2 Spectral Graph Theory

We'll see here two of the most celebrated algorithms stemming from Spectral Graph Theory:
1. [Spectral clustering](https://en.wikipedia.org/wiki/Spectral_clustering): clustering data using a similarity graphs
1. [Laplacian eigenmaps](https://en.wikipedia.org/wiki/Nonlinear_dimensionality_reduction#Laplacian_eigenmaps): dimensionality reduction which preserves geodesic distances

**TODO**: look at <https://nbviewer.jupyter.org/github/mdeff/ntds_2017/blob/outputs/assignments/03_solution.ipynb>

**TODO**: maybe look at spectrum and eigengap

## 2.1 Spectral clustering

**TODO**:
* mathematical formulation of the problem
* solution is the first eigenvectors of the Laplacian
* implement with pygsp
* need data with labels: FMA?

## 2.2 Laplacian eigenmaps

**TODO**:
* mathematical formulation of the problem
* solution is the first eigenvectors of the Laplacian
* implement with pygsp
* need data with labels: FMA?